## **Santander Dev Week 2025 (Criando Banco Vetorial)**

- **Contexto**: Para aperfeçoar a resposta do agente, deve ser criado um banco vetorial que armazenar um catálogo de investimentos referentes a recomendação de investimentos por perfil de investidor

**Cientista de Dados**: Lucca de Sena Barbosa

#### **Configurando Parâmetros e Importando Libs:**

In [ ]:
import fitz 
import os
from dotenv import load_dotenv, find_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

# Carrega as variáveis de ambiente
load_dotenv(find_dotenv())
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Configurações para divisão de texto e carregamento do PDF
overlap_size = 200 
chunk_size = 500
file_path = "doc/catalogo-investimentos-santander-simulacao.pdf"

# Abre o documento PDF
docs = fitz.open(file_path)

# Embeddings e Text Splitter
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004", google_api_key=GOOGLE_API_KEY)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=overlap_size
)

#### **Extrai o texto do PDF e divide em chunks**

In [ ]:
documents = []
for doc in docs:
    text = doc.get_text()
    splits = text_splitter.split_text(text)
    for split in splits:
        documents.append({"page_content": split, "metadata": {"page_number": doc.number}})

#### **Cria o banco vetorial  e salva localmente**

In [ ]:

vector_store = FAISS.from_documents(documents, embeddings)
vector_store.save_local("db")

#### **Teste para conferir o Retorno**

In [24]:
perfil_investidor = "Moderado"

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

docs = retriever.invoke(perfil_investidor)


In [25]:
context = ""
for doc in docs:
    context += doc.page_content 
    context += "\n\n"

In [26]:
display(context)

'Rentabilidade: 94% do CDI (Equivalente a um CDB de 115% do CDI, devido à isenção\nfiscal).\nRisco: Baixo (Garantia do FGC).\nLiquidez: No vencimento (Carência de 90 dias mínima, vencimento em 1 ano).\nAplicação Mínima: R$ 1.000,00.\n2. PERFIL MODERADO\nDefinição: Busca superar a inflação e a poupança. Aceita pequenas oscilações de curto prazo\nem troca de retornos médios mais atrativos no médio prazo.\nProduto A: Santander Select Multimercado Macro\nCategoria: Fundo Multimercado\n\nem troca de retornos médios mais atrativos no médio prazo.\nProduto A: Santander Select Multimercado Macro\nCategoria: Fundo Multimercado\nDescrição: Um fundo gerido por especialistas que aloca recursos em juros, moedas e\nbolsa, adaptando-se ao cenário econômico. Busca proteger o patrimônio em quedas e\nganhar nas altas.\n30/12/2025, 16:23\nGoogle Gemini\nhttps://gemini.google.com/app/39d01efbed664be1?hl=pt-BR\n1/2\n\npadrão. O foco é a geração de renda passiva mensal (dividendos) isenta de IR.\nRentabilid